In [6]:
import numpy as np
import glob
import pandas as pd

In [282]:
#step 1 - combining all daily files into one file
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

In [406]:
# setting df as a DataFrame as combined_csv
#all data from the .csv is in strings it appears
df = pd.read_csv("combined_csv.csv")

#drop blanks from the ticker
df['ticker'] = df['ticker'].str.strip()
df['ticker'].replace('', np.nan, inplace=True)
df['ticker'].replace('TRUE', np.nan, inplace=True)
#drop ticker.str.len() > 6 from ticker (no ticker is longer than 6)
df['ticker'].values[df['ticker'].str.len() > 6,] = np.nan
df.dropna(subset=['ticker'], inplace=True)

#drop blanks and 0s from price
df['price'] = df['price'].str.strip() #strip blank spaces
df['price'].replace('', np.nan, inplace=True) #replace blanks with NaN
df['price'] = df['price'].replace(',','', regex=True) #strip commas
df['price'] = pd.to_numeric(df['price']) #convert price column from string to numbers
df['price'].replace(0,np.nan,inplace=True) #remove 0s
df.dropna(subset=['price'], inplace=True) #remove all NaNs

#converting num_mentions, pos_count, neg_count, price, price_change_net into int/floats
df['num_mentions'] = pd.to_numeric(df['num_mentions'])
df['pos_count'] = pd.to_numeric(df['pos_count'])
df['neg_count'] = pd.to_numeric(df['neg_count'])
df['price'] = pd.to_numeric(df['price'])
df['price_change_net'] = df['price_change_net'].replace(',','', regex=True) #strip commas
df['price_change_net'] = pd.to_numeric(df['price_change_net'])

#converting date column into date_time format
df['date'] = pd.to_datetime(df['date'])

#cleaning pct_mentions and converting to number
df['pct_mentions'] = df['pct_mentions'].str.strip() #strip blank spaces
df['pct_mentions'] = df['pct_mentions'].replace('<1','0.5', regex=True) #change '<1' to '0.5'
df['pct_mentions'] = pd.to_numeric(df['pct_mentions'])

In [434]:
df['price_change_pct'] = df['price_change_pct'].replace('%','', regex=True) #strip %
df['price_change_pct'] = df['price_change_pct'].replace(',','', regex=True) #strip ,
df = df.replace('nan%', np.nan).dropna(subset=['price_change_pct'])
df['price_change_pct'] = df['price_change_pct'].str.strip() #strip blank spaces
df['price_change_pct'] = pd.to_numeric(df['price_change_pct']) #convert to integer

ValueError: Unable to parse string "+undefined" at position 5290

In [431]:
pd.set_option('display.max_rows', 50000)

#counts.to_csv('test.csv')

In [433]:
df.price_change_pct.value_counts()

0.00          540
+0.20         133
+0.10         133
+0.12         130
-0.20         125
-0.07         123
-0.08         123
-0.10         122
+0.36         119
+0.17         116
-0.53         114
+0.13         113
-0.04         113
+0.11         112
+0.28         110
+0.09         110
+0.30         109
+0.08         106
+0.41         106
-0.29         105
-0.27         105
+0.05         105
+0.02         105
-0.02         104
-0.18         104
+0.16         104
-0.13         104
+0.04         103
+0.22         103
+0.06         103
+0.21         102
+0.35         102
+0.24         102
-0.22         102
-0.06         101
-0.19         101
-0.23         101
-0.03         101
+0.38         101
-0.16         100
+0.23         100
+0.37         100
-0.31          99
+0.34          99
+0.33          98
-0.26          98
-0.14          98
-0.25          98
+0.07          98
-0.55          97
-0.05          97
-0.17          97
+0.43          97
-0.21          97
-0.47          96
-0.44     

In [428]:
df[df.price_change_pct == 'NaN%'].price_change_pct

Series([], Name: price_change_pct, dtype: object)

In [371]:
print(df.loc[[5290]])

     ticker       date                                      url  num_mentions  \
5290     OI 2021-01-19  https://finance.yahoo.com/quote/OI?p=OI             5   

     pct_mentions  pos_count  neg_count  bullish_pct  bearish_pct  \
5290           <1          0          0            0            0   

      neutral_pct  price  price_change_net price_change_pct  \
5290          100  12.69             -0.27            -2.08   

              time_of_price  
5290  At close:  4:00PM EST  


In [357]:
df.sort_values('price_change_pct')

,ticker,date,url,num_mentions,pct_mentions,pos_count,neg_count,bullish_pct,bearish_pct,neutral_pct,price,price_change_net,price_change_pct,time_of_price
10453,LOGI,2021-02-05,https://finance.yahoo.com/quote/LOGI?p=LOGI,1,<1,0,0,0,0,100,109.70,109.70,+-0.06%,At close: 4:00PM EST
10434,XRT,2021-02-05,https://finance.yahoo.com/quote/XRT?p=XRT,1,<1,1,0,100,0,0,77.22,77.22,+-0.09%,At close: 3:59PM EST
10455,COST,2021-02-05,https://finance.yahoo.com/quote/COST?p=COST,1,<1,1,0,100,0,0,355.85,355.85,+-0.10%,At close: 4:00PM EST
10478,GEM,2021-02-05,https://finance.yahoo.com/quote/GEM?p=GEM,2,<1,0,0,0,0,100,40.69,40.69,+-0.17%,At close: 4:00PM EST
10474,BOX,2021-02-05,https://finance.yahoo.com/quote/BOX?p=BOX,1,<1,1,0,100,0,0,18.50,18.50,+-0.33%,At close: 4:00PM EST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52033,WKHS,2021-09-22,https://finance.yahoo.com/quote/WKHS?p=WKHS,1,<1,0,1,0,100,0,8.19,0.00,0.00%,At close: 4:00PM EDT
26155,INDA,2021-04-12,https://finance.yahoo.com/quote/INDA?p=INDA,1,<1,0,1,0,100,0,41.69,0.00,0.00%,At close: 4:00PM EDT
28080,TDA,2021-04-22,https://finance.yahoo.com/quote/TDA?p=TDA,2,<1,0,2,0,100,0,25.58,0.00,0.00%,At close: 3:59PM EDT
35011,SHIP,2021-06-03,https://finance.yahoo.com/quote/SHIP?p=SHIP,2,<1,0,0,0,0,100,1.05,0.00,0.00%,At close: 4:00PM EDT


In [405]:
df.dtypes

ticker                      object
date                datetime64[ns]
url                         object
num_mentions                 int64
pct_mentions               float64
pos_count                    int64
neg_count                    int64
bullish_pct                  int64
bearish_pct                  int64
neutral_pct                  int64
price                      float64
price_change_net           float64
price_change_pct            object
time_of_price               object
dtype: object